Evaluate google translate

In [6]:
from pathlib import Path
import sys
root = Path.cwd().parent
sys.path.append(str(root))

from google.cloud import translate_v2 as gg_translate
import pandas as pd
import os
#os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = YOUR_CREDENTIALS

In [7]:
sentences = pd.read_csv('CWQ Annotation - EN.csv')
sentences = list(sentences['questionWithBrackets'].dropna())

In [8]:
translate_client = gg_translate.Client()
def translate_with_qmark(q, target):
    q = q if q[-1] == "?" else q + "?"
    q_trans = translate_client.translate(q, target_language=target)["translatedText"]
    return q_trans[:-1] if q_trans[-1] == "?" else q_trans

In [10]:
jp = []
for en in sentences:
    jp.append(translate_with_qmark(en, 'ja'))

In [12]:
dic = [{'questionWithBrackets':en, 'questionWithBrackets_jp_gg':jp} for en,jp in zip(sentences, jp)]

In [13]:
# write a file for manual assessment
import csv
with open('CWQ Annotation - JP - Google.csv','w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=['questionWithBrackets', 'questionWithBrackets_jp_gg'])
    writer.writeheader()
    writer.writerows(dic)

In [16]:
# reference-based assessment
from sacrebleu.metrics import BLEU
from sacrebleu.tokenizers import tokenizer_ja_mecab

In [18]:
sentences = pd.read_csv('CWQ Annotation - GOLD-JP.csv')
sentences_en = list(sentences['questionPatternModEntities'].dropna())
gold_jp = list(sentences['questionPatternModEntitiesJP'].dropna())

In [19]:
infer_jp = []
for en in sentences_en:
    infer_jp.append(translate_with_qmark(en, 'ja'))

In [20]:
bleu = BLEU(tokenize='ja-mecab')
bleu.corpus_score(hypotheses=infer_jp, references=[gold_jp])

BLEU = 42.36 76.9/53.7/38.4/26.8 (BP = 0.933 ratio = 0.935 hyp_len = 3266 ref_len = 3494)

BLEU = 42.36 76.9/53.7/38.4/26.8 (BP = 0.933 ratio = 0.935 hyp_len = 3266 ref_len = 3494)

In [34]:
def rep(sen):
    sen = sen.replace('だった','')
    sen = sen.replace('？','')
    sen = sen.replace('でしたか','')
    sen = sen.replace('ですか','')
    sen = sen.replace('によって','に')
    sen = sen.replace(' ','')
    return sen
new_infer_jp = []
new_gold_jp = []
for inf, gol in zip(infer_jp, gold_jp):
    new_infer_jp.append(rep(inf))
    new_gold_jp.append(rep(gol))



In [35]:
bleu = BLEU(tokenize='ja-mecab')
bleu.corpus_score(hypotheses=new_infer_jp, references=[new_gold_jp])

BLEU = 45.07 79.0/56.7/41.4/29.3 (BP = 0.934 ratio = 0.936 hyp_len = 3102 ref_len = 3315)